In [1]:
%run ./process_data_new.py

:::::::::::::::::::: PREPARING ::::::::::::::::::::
[INFO]: Featurizing 45 files: 
  Featurizing 1.mp3 -> .........................DONE
  Featurizing 10.mp3 -> .........................DONE
  Featurizing 11.mp3 -> .........................DONE
  Featurizing 12.mp3 -> .........................DONE
  Featurizing 13.mp3 -> .........................DONE
  Featurizing 14.mp3 -> .........................DONE
  Featurizing 15.mp3 -> .........................DONE
  Featurizing 16.mp3 -> .........................DONE
  Featurizing 17.mp3 -> .........................DONE
  Featurizing 18.mp3 -> .........................DONE
  Featurizing 19.mp3 -> .........................DONE
  Featurizing 2.mp3 -> .........................DONE
  Featurizing 20.mp3 -> .........................DONE
  Featurizing 21.mp3 -> .........................DONE
  Featurizing 22.mp3 -> .........................DONE
  Featurizing 23.mp3 -> .........................DONE
  Featurizing 24.mp3 -> .........................DONE
 

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
import joblib
import random
import gc
import matplotlib.pyplot as plt

2025-07-30 21:15:08.637696: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


In [4]:
df = pd.read_hdf("./dataset.h5", key="df")
df

,Cens_C,Cens_Db,Cens_D,Cens_Eb,Cens_E,Cens_F,Cens_Gb,Cens_G,Cens_Ab,Cens_A,Cens_Bb,Cens_B,chord,filename
0,0.458163,0.228498,0.335618,0.385245,0.280831,0.562060,0.101404,0.142517,0.211683,0.000000,0.081103,0.007991,FMin7,../Datasets/1.mp3
1,0.456882,0.228481,0.329903,0.386718,0.270461,0.563033,0.104248,0.142325,0.223759,0.000000,0.096144,0.008832,FMin7,../Datasets/1.mp3
2,0.454308,0.229081,0.324611,0.388377,0.260669,0.563478,0.107217,0.142526,0.235052,0.000000,0.110957,0.009594,FMin7,../Datasets/1.mp3
3,0.450574,0.230340,0.319815,0.390179,0.251621,0.563336,0.110333,0.143008,0.245606,0.000000,0.125269,0.010269,FMin7,../Datasets/1.mp3
4,0.445894,0.232287,0.315578,0.392097,0.243367,0.562581,0.113524,0.143594,0.255459,0.000000,0.138864,0.010856,FMin7,../Datasets/1.mp3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,0.207725,0.076254,0.397630,0.012440,0.081124,0.025550,0.113143,0.602074,0.253705,0.415162,0.012520,0.416366,AMin7,../Datasets/9.mp3
109,0.200846,0.072857,0.392787,0.013882,0.080756,0.028333,0.115661,0.609691,0.257939,0.416193,0.013882,0.409283,AMin7,../Datasets/9.mp3
110,0.194574,0.068975,0.387773,0.015407,0.080133,0.031256,0.117944,0.616832,0.261322,0.417602,0.015407,0.402578,AMin7,../Datasets/9.mp3
111,0.188929,0.064633,0.382609,0.017022,0.079286,0.034334,0.119918,0.623479,0.263771,0.419261,0.017022,0.396526,AMin7,../Datasets/9.mp3


In [5]:
unique_file = df["filename"].unique()
train_file, test_file = sk.model_selection.train_test_split(
    unique_file,
    random_state=42,
    test_size=0.2,
)
test_file, val_file = sk.model_selection.train_test_split(
    train_file,
    random_state=42,
    test_size=0.5,
)

df_train = df[df["filename"].isin(train_file)]
df_val = df[df["filename"].isin(val_file)]
df_test = df[df["filename"].isin(test_file)]

In [6]:
print("Chords")
print("\n## TRAIN ##", df_train["chord"].value_counts())
print("\n## VAL ##", df_val["chord"].value_counts())
print("\n## TEST ##", df_test["chord"].value_counts())

Chords

## TRAIN ## chord
AbMin7    15671
CMin7     15642
FMin7     15555
GbMin7    15474
DbMin7    15298
EMin7     14917
AMin7     14859
BbMin7    14805
GMin7     14679
EbMin7    14666
DMin7     14658
BMin7     14601
AMaj7     14174
AbMaj7    13936
EbMaj7    13895
EMaj7     13889
BbMaj7    13815
GbMaj7    13661
BMaj7     13602
GMaj7     13564
DbMaj7    13555
DMaj7     13547
CMaj7     13523
FMaj7     13339
G7         3454
C7         3395
Bb7        3386
B7         3360
Db7        3232
Ab7        3230
D7         3200
Eb7        3185
E7         3175
A7         3162
Gb7        3155
F7         3141
Name: count, dtype: int64

## VAL ## chord
AMaj7     9935
EbMaj7    9907
EMaj7     9867
BbMaj7    9819
AbMaj7    9812
BMaj7     9731
GMaj7     9713
GbMaj7    9658
DMaj7     9651
CMaj7     9531
DbMaj7    9529
FMaj7     9472
AbMin7    9040
CMin7     9006
FMin7     8968
GbMin7    8910
DbMin7    8722
AMin7     8683
EMin7     8678
BbMin7    8492
GMin7     8455
EbMin7    8426
BMin7     8409
DMin7     

In [7]:
# y = df["chord"]
# X = df.drop(columns="chord")
# encoder = sk.preprocessing.LabelEncoder()
# y_encoded = encoder.fit_transform(y)

# joblib.dump(encoder, "./encoder.xz")

def get_Xy(df: pd.DataFrame):
    return df.drop(["chord", "filename"], axis=1), df["chord"]

X_train, y_train = get_Xy(df_train)
X_val, y_val = get_Xy(df_val)
X_test, y_test = get_Xy(df_test)

encoder = sk.preprocessing.LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_val_encoded = encoder.transform(y_val)
y_test_encoded = encoder.transform(y_test)

joblib.dump(encoder, "./encoder.xz")

['./encoder.xz']

In [8]:
# del df
# gc.collect()

# LSTM

In [9]:
# SEQUENCE_LEN = 20 # 0.1 * 20.0 = 2 sec of sequence data

# def make_seq(X, y_encoded):
#     X_seq, y_encoded_seq = None, None
#     X_seq_list = []
#     y_encoded_seq_list = []
#     for i in range(len(X) - SEQUENCE_LEN + 1):
#         X_seq_list.append(X.values[i : i + SEQUENCE_LEN, :])
#         y_encoded_seq_list.append(y_encoded[i + SEQUENCE_LEN - 1])
    
#     return np.array(X_seq_list), np.array(y_encoded_seq_list)

# X_train, y_train_encoded = make_seq(X_train, y_train_encoded)
# X_val, y_val_encoded = make_seq(X_val, y_val_encoded)
# X_test, y_test_encoded = make_seq(X_test, y_test_encoded)

# print("TRAIN")
# print("X sequence shape: ", X_train.shape)
# print("y sequence shape: ", y_train_encoded.shape)

# print("\nVAL")
# print("X sequence shape: ", X_val.shape)
# print("y sequence shape: ", y_val_encoded.shape)

# print("\nTEST")
# print("X sequence shape: ", X_test.shape)
# print("y sequence shape: ", y_test_encoded.shape)

# gc.collect()

In [10]:
# X_seq_train, X_seq_test, y_seq_train, y_seq_test = sk.model_selection.train_test_split(
#     X_seq,
#     y_encoded_seq,
#     test_size=0.2,
#     random_state=42,
#     stratify=y_encoded_seq,
# )

# print(f"X_train: {len(X_seq_train)}")
# print(f"y_train: {len(y_seq_train)}")
# print(f"X_test:  {len(X_seq_test)}")
# print(f"y_test:  {len(y_seq_test)}")

# del X_seq
# del y_encoded_seq
# gc.collect()

In [11]:
# print("Total feature: ", X_train.shape[2])
# print("Total class:   ", len(encoder.classes_))

In [12]:
class_weights = sk.utils.class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train,
)
class_weight_dict = dict(enumerate(class_weights))

In [13]:
layers = [
    (16, 16),
    (32, 32),
    (64, 64)
]

seq_lens = [5, 10, 20]
    
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True,
    # monitor="val_accuracy"
)

def make_seq(X, y_encoded, seq_len):
    X_seq, y_encoded_seq = None, None
    X_seq_list = []
    y_encoded_seq_list = []
    for i in range(len(X) - seq_len + 1):
        X_seq_list.append(X.values[i : i + seq_len, :])
        y_encoded_seq_list.append(y_encoded[i + seq_len - 1])
    
    return np.array(X_seq_list), np.array(y_encoded_seq_list)

# models = []
for seq_len in seq_lens:
    X_train_seq, y_train_encoded_seq = make_seq(X_train, y_train_encoded, seq_len)
    X_val_seq, y_val_encoded_seq = make_seq(X_val, y_val_encoded, seq_len)
    X_test_seq, y_test_encoded_seq = make_seq(X_test, y_test_encoded, seq_len)
    
    for layer_1, layer_2 in layers:
        model_lstm = tf.keras.Sequential([
            # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(SEQUENCE_LEN, X_seq_train.shape[2]))),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
                layer_1,
                return_sequences=True,
                input_shape=(seq_len, X_train_seq.shape[2]),
                kernel_regularizer=tf.keras.regularizers.l2(1e-3),
                recurrent_regularizer=tf.keras.regularizers.l2(1e-3),
                recurrent_dropout=0.25,
            )),
            tf.keras.layers.Dropout(0.25),
                
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
                layer_2,
                return_sequences=False,
                kernel_regularizer=tf.keras.regularizers.l2(1e-3),
                recurrent_regularizer=tf.keras.regularizers.l2(1e-3),
                recurrent_dropout=0.25,
            )),
            tf.keras.layers.Dropout(0.25),
        
            tf.keras.layers.Dense(
                len(encoder.classes_),
                activation='softmax',
                # kernel_regularizer=tf.keras.regularizers.l2(1e-2)
            ),
        ])
        
        model_lstm.compile(
            optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
        )
        
        # model_lstm.summary()
        
        history = model_lstm.fit(
            # X_seq_train,
            # y_seq_train,
            X_train_seq,
            y_train_encoded_seq,
            epochs=500,
            batch_size=32,
            # validation_split=0.1,
            validation_data=(X_val_seq, y_val_encoded_seq),
            verbose=1,
            callbacks=[early_stopping],
            class_weight=class_weight_dict,
        )
    
        # models.append(model_lstm)
        
        model_lstm.save(f"./Models/model_lstm_{layer_1}_{layer_2}_seq_{seq_len}.keras")
        pd.DataFrame(history.history).to_csv(f"./History/model_lstm_{layer_1}_{layer_2}_seq_{seq_len}_history.csv", index=False)

        print(f"::::: model_lstm_{layer_1}_{layer_2}_seq_{seq_len} :::::")
        print(model_lstm.evaluate(X_test_seq, y_test_encoded_seq))
        y_pred = np.argmax(model_lstm.predict(X_test_seq), axis=1)
        print(
            sk.metrics.classification_report(y_test_encoded_seq, y_pred, target_names=encoder.classes_)
        )
        print("")
    
        gc.collect()

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 100s 8ms/step - accuracy: 0.4063 - loss: 2.2756 - val_accuracy: 0.5487 - val_loss: 1.6238
Epoch 2/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 93s 8ms/step - accuracy: 0.5085 - loss: 1.7623 - val_accuracy: 0.5500 - val_loss: 1.5868
Epoch 3/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 93s 8ms/step - accuracy: 0.5161 - loss: 1.7189 - val_accuracy: 0.5611 - val_loss: 1.5509
Epoch 4/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 92s 8ms/step - accuracy: 0.5240 - loss: 1.6945 - val_accuracy: 0.5648 - val_loss: 1.5316
Epoch 5/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 92s 8ms/step - accuracy: 0.5267 - loss: 1.6843 - val_accuracy: 0.5605 - val_loss: 1.5333
Epoch 6/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 89s 7ms/step - accuracy: 0.5280 - loss: 1.6794 - val_accuracy: 0.5672 - val_loss: 1.5186
Epoch 7/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 94s 8ms/step - accuracy: 0.5316 - loss: 1.6682 - val_accuracy: 0.5587 - val_loss: 1.5372
Epoch 8/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 95s 8ms/step - a

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 106s 8ms/step - accuracy: 0.4611 - loss: 2.0833 - val_accuracy: 0.5647 - val_loss: 1.5762
Epoch 2/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 100s 8ms/step - accuracy: 0.5433 - loss: 1.6444 - val_accuracy: 0.5802 - val_loss: 1.5085
Epoch 3/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 101s 8ms/step - accuracy: 0.5542 - loss: 1.5894 - val_accuracy: 0.5699 - val_loss: 1.5288
Epoch 4/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 102s 8ms/step - accuracy: 0.5581 - loss: 1.5676 - val_accuracy: 0.5853 - val_loss: 1.4728
Epoch 5/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 101s 8ms/step - accuracy: 0.5607 - loss: 1.5489 - val_accuracy: 0.5904 - val_loss: 1.4577
Epoch 6/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 101s 8ms/step - accuracy: 0.5629 - loss: 1.5399 - val_accuracy: 0.6002 - val_loss: 1.4155
Epoch 7/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 101s 8ms/step - accuracy: 0.5643 - loss: 1.5291 - val_accuracy: 0.5936 - val_loss: 1.4393
Epoch 8/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 103s 9ms/s

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

12012/12012 ━━━━━━━━━━━━━━━━━━━━ 219s 18ms/step - accuracy: 0.3683 - loss: 2.4067 - val_accuracy: 0.5258 - val_loss: 1.6938
Epoch 2/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 212s 18ms/step - accuracy: 0.4970 - loss: 1.8052 - val_accuracy: 0.5401 - val_loss: 1.6271
Epoch 3/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 212s 18ms/step - accuracy: 0.5086 - loss: 1.7394 - val_accuracy: 0.5581 - val_loss: 1.5667
Epoch 4/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 212s 18ms/step - accuracy: 0.5147 - loss: 1.7087 - val_accuracy: 0.5565 - val_loss: 1.5567
Epoch 5/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 213s 18ms/step - accuracy: 0.5220 - loss: 1.6856 - val_accuracy: 0.5591 - val_loss: 1.5411
Epoch 6/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 213s 18ms/step - accuracy: 0.5249 - loss: 1.6752 - val_accuracy: 0.5630 - val_loss: 1.5355
Epoch 7/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 213s 18ms/step - accuracy: 0.5246 - loss: 1.6691 - val_accuracy: 0.5640 - val_loss: 1.5264
Epoch 8/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 214s 18ms/step 

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 235s 19ms/step - accuracy: 0.4208 - loss: 2.2246 - val_accuracy: 0.5627 - val_loss: 1.5923
Epoch 2/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 240s 20ms/step - accuracy: 0.5355 - loss: 1.6787 - val_accuracy: 0.5771 - val_loss: 1.5250
Epoch 3/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 241s 20ms/step - accuracy: 0.5502 - loss: 1.6100 - val_accuracy: 0.5857 - val_loss: 1.4934
Epoch 4/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 244s 20ms/step - accuracy: 0.5573 - loss: 1.5807 - val_accuracy: 0.5933 - val_loss: 1.4663
Epoch 5/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 255s 21ms/step - accuracy: 0.5608 - loss: 1.5607 - val_accuracy: 0.5871 - val_loss: 1.4712
Epoch 6/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 262s 22ms/step - accuracy: 0.5647 - loss: 1.5415 - val_accuracy: 0.5970 - val_loss: 1.4475
Epoch 7/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 263s 22ms/step - accuracy: 0.5662 - loss: 1.5337 - val_accuracy: 0.5959 - val_loss: 1.4485
Epoch 8/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 267

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 303s 25ms/step - accuracy: 0.4519 - loss: 2.1537 - val_accuracy: 0.5838 - val_loss: 1.5825
Epoch 2/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 299s 25ms/step - accuracy: 0.5567 - loss: 1.6550 - val_accuracy: 0.5941 - val_loss: 1.5321
Epoch 3/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 297s 25ms/step - accuracy: 0.5692 - loss: 1.5943 - val_accuracy: 0.5986 - val_loss: 1.5023
Epoch 4/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 320s 27ms/step - accuracy: 0.5766 - loss: 1.5579 - val_accuracy: 0.6088 - val_loss: 1.4775
Epoch 5/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 329s 27ms/step - accuracy: 0.5786 - loss: 1.5465 - val_accuracy: 0.6159 - val_loss: 1.4559
Epoch 6/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 330s 27ms/step - accuracy: 0.5854 - loss: 1.5258 - val_accuracy: 0.6164 - val_loss: 1.4528
Epoch 7/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 327s 27ms/step - accuracy: 0.5870 - loss: 1.5191 - val_accuracy: 0.6168 - val_loss: 1.4487
Epoch 8/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 330